In [2]:
%load_ext autoreload
%autoreload 2

import sys
import os

import pandas as pd
import joblib
import json
import numpy as np

sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
folder_to_cluster_name = {
    "type": {
        "00004": "70_limit",
        "00009": "dont_overtake",
        "00035": "go_straight",
        "00038": "go_right",
    },
    "colour": {
        "00004": "red",
        "00009": "red",
        "00035": "blue",
        "00038": "blue",
    }
}

In [4]:
data_dir = "/mnt/data/stephana93dm/storage/projects/alternative_clustering/alternative_clustering/datasets/"

dataset_name = "GTSRB"
load_dir = os.path.join(data_dir, dataset_name, "train")
save_dir = os.path.join(data_dir, "processed")


In [5]:
df = []

for folder in os.listdir(load_dir):
    if not os.path.isdir(os.path.join(load_dir, folder)):
        continue
    colour_ = folder_to_cluster_name["colour"][folder]
    type_ = folder_to_cluster_name["type"][folder]

    for file in os.listdir(os.path.join(load_dir, folder)):
        if file.endswith(".ppm"):
            df.append({
                "name": f"{type_}_{file}",
                "image_filename": os.path.join(load_dir, folder, file),
                "colour": colour_,
                "type": type_,
            })

df = pd.DataFrame(df)
df.head()

,name,image_filename,colour,type
0,go_straight_00000_00011.ppm,/mnt/data/stephana93dm/storage/projects/altern...,blue,go_straight
1,go_straight_00032_00026.ppm,/mnt/data/stephana93dm/storage/projects/altern...,blue,go_straight
2,go_straight_00022_00013.ppm,/mnt/data/stephana93dm/storage/projects/altern...,blue,go_straight
3,go_straight_00038_00020.ppm,/mnt/data/stephana93dm/storage/projects/altern...,blue,go_straight
4,go_straight_00015_00017.ppm,/mnt/data/stephana93dm/storage/projects/altern...,blue,go_straight


In [6]:
len(df)

6720

In [7]:
from sklearn.preprocessing import LabelEncoder

type_le = LabelEncoder()
colour_le = LabelEncoder()

df["type_clusters"] = type_le.fit_transform(df["type"])
df["colour_clusters"] = colour_le.fit_transform(df["colour"])

In [8]:
cluster_to_name = {
    "type": {i: name for i, name in enumerate(type_le.classes_)},
    "colour": {i: name for i, name in enumerate(colour_le.classes_)}
}
cluster_to_name

{'type': {0: '70_limit', 1: 'dont_overtake', 2: 'go_right', 3: 'go_straight'},
 'colour': {0: 'blue', 1: 'red'}}

In [20]:
import shutil


save_folder = os.path.join(save_dir, dataset_name)
image_save_folder = os.path.join(save_folder, "images")
clustering_save_folder = os.path.join(save_folder, "clustering")
os.makedirs(image_save_folder, exist_ok=True)
os.makedirs(clustering_save_folder, exist_ok=True)

with open(os.path.join(save_folder, "cluster_to_name.json"), "w") as f:
    json.dump(cluster_to_name, f)

for cluster_name in ["type", "colour"]:
    dictionary = {}
    for idx, row in df.iterrows():
        name = row["name"]
        dictionary[name] = row[f"{cluster_name}_clusters"]

        src = row["image_filename"]
        dst = os.path.join(image_save_folder, name)
        shutil.copyfile(src, dst)
    
    save_file = os.path.join(clustering_save_folder, f"clusters_{cluster_name}.json")
    with open(save_file, "w") as f:
        json.dump(dictionary, f)